In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from PIL import Image
import matplotlib.pyplot as plt
from pathlib import Path
import os
import pickle
import random
import requests
import io
import zipfile
from scipy.stats import norm
import pypfopt
import cvxpy as cp

#!pip install PyPortfolioOpt
from pypfopt import EfficientCVaR, EfficientFrontier
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import sample_cov


# Caveats
- daily to monthly
- monhtly optimization to daily evaulation
- objective function  and output mismatch during evaulation ( see Vol_managed discussion)

In [3]:
# DAILY DATA
industry_mkt = pd.read_csv('industry_mkt.csv')
industry = industry_mkt[[c for c in industry_mkt.columns if c !='Mkt-RF']]


# Date index
date_col = next((c for c in ["date", "Date", "DATE"] if c in industry.columns), None)

industry[date_col] = pd.to_datetime(industry[date_col])
industry = industry.set_index(date_col).sort_index()
industry.head()

C:\Users\95att\AppData\Local\Temp\ipykernel_4268\2192586430.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  industry[date_col] = pd.to_datetime(industry[date_col])


,Durbl,Enrgy,HiTec,Hlth,Manuf,NoDur,Other,Shops,Telcm,Utils
date,,,,,,,,,,
2015-12-14,-0.0028,0.0076,0.0050,0.0040,-0.0016,0.0054,-0.0005,0.0077,0.0003,0.0001
2015-12-15,0.0097,0.0291,0.0041,0.0173,0.0035,0.0089,0.0193,-0.0006,0.0119,0.0083
2015-12-16,0.0266,-0.0070,0.0140,0.0152,0.0141,0.0183,0.0159,0.0153,0.0189,0.0230
2015-12-17,-0.0222,-0.0244,-0.0147,-0.0135,-0.0149,-0.0125,-0.0150,-0.0145,-0.0137,-0.0040
2015-12-18,-0.0131,-0.0140,-0.0183,-0.0131,-0.0169,-0.0166,-0.0221,-0.0102,-0.0187,-0.0131


In [4]:
# Portfolio evaulation, similar to JULIA table

def df_to_booktabs_latex(df: pd.DataFrame, caption=None, label=None) -> str:
    latex = df.to_latex(
        escape=True,
        float_format=lambda x: f"{x:.2f}",
        column_format="l" + "r"*df.shape[1],
        bold_rows=False
    )
    # convert to booktabs style
    latex = latex.replace("\\toprule", "\\toprule").replace("\\midrule", "\\midrule").replace("\\bottomrule", "\\bottomrule")
    if caption or label:
        # wrap in table environment if requested
        body = latex
        lines = ["\\begin{table}[!htbp]", "\\centering"]
        if caption:
            lines.append(f"\\caption{{{caption}}}")
        if label:
            lines.append(f"\\label{{{label}}}")
        lines.append(body.strip())
        lines.append("\\end{table}")
        latex = "\n".join(lines)
    return latex
def portfolio_stats_paper_style(returns,
                                periods_per_year=252,
                                rf_annual=0.0,
                                target=0.0,
                                alpha=0.95):

    r = pd.Series(returns).dropna().astype(float).to_numpy()
    if len(r) < 2:
        raise ValueError("need at least 2 observations")
    if not (0.0 < alpha < 1.0):
        raise ValueError("alpha must be in (0,1)")

    mu = float(np.mean(r))
    sigma2 = float(np.var(r, ddof=0))
    sigma = float(np.sqrt(sigma2))

    mu_ann = mu * periods_per_year
    sigma2_ann = sigma2 * periods_per_year
    sigma_ann = float(np.sqrt(sigma2_ann))

    # downside RMS / LPM2
    downside = (r[r < target] - target)
    semivar = 0.0 if downside.size == 0 else float(np.mean(downside**2))
    semidev_ann = float(np.sqrt(semivar * periods_per_year))

    # VaR/CVaR
    q = 1.0 - alpha
    VaR = float(np.quantile(r, q))
    tail = r[r <= VaR]
    CVaR = VaR if tail.size == 0 else float(np.mean(tail))

    # drawdowns
    wealth = np.concatenate([[1.0], np.cumprod(1.0 + r)])
    peak = np.maximum.accumulate(wealth)
    dd = 1.0 - wealth / peak
    pos_dd = dd[dd > 0]
    avg_dd = 0.0 if pos_dd.size == 0 else float(np.mean(pos_dd))

    # excess mean (per-period rf from annual rf)
    rf_per = (1.0 + rf_annual)**(1.0 / periods_per_year) - 1.0
    excess_ann_mean = (mu - rf_per) * periods_per_year

    sharpe = np.nan if sigma_ann == 0 else float(excess_ann_mean / sigma_ann)
    sortino = np.nan if semidev_ann == 0 else float(excess_ann_mean / semidev_ann)

    # tail-adjusted Sharpe (NO annualization of CVaR/mVaR)
    ta_sharpe_cvar = np.nan if CVaR == 0 else float(excess_ann_mean / abs(CVaR))

    # Cornish-Fisher modified VaR
    if sigma == 0:
        skew = 0.0
        exkurt = 0.0
    else:
        xc = r - mu
        m3 = float(np.mean(xc**3))
        m4 = float(np.mean(xc**4))
        skew = m3 / (sigma**3)
        kurt = m4 / (sigma**4)
        exkurt = kurt - 3.0

    z = float(norm.ppf(q))
    z_cf = (z
            + (1/6)  * (z**2 - 1)   * skew
            + (1/24) * (z**3 - 3*z) * exkurt
            - (1/36) * (2*z**3 - 5*z) * (skew**2))

    mVaR = float(mu + sigma * z_cf)
    ta_sharpe_mvar = np.nan if mVaR == 0 else float(excess_ann_mean / abs(mVaR))

    return {
        "Ann. Mean (%)": 100 * mu_ann,
        "Ann. StdDev (%)": 100 * sigma_ann,
        "Ann. SemiDev (%)": 100 * semidev_ann,
        "CVaR 95% (%)": 100 * CVaR,
        "Avg DD (%)": 100 * avg_dd,
        "VaR 95% (%)": 100 * VaR,
        "Sharpe (ann.)": sharpe,
        "Sortino (ann.)": sortino,
        "Tail-Adj Sharpe (CVaR95)": ta_sharpe_cvar,
        "Tail-Adj Sharpe (mVaR95)": ta_sharpe_mvar,
    }


def make_table_for_portfolios(portfolios: dict,
                              periods_per_year=252,
                              rf_annual=0.0,
                              target=0.0,
                              alpha=0.95) -> pd.DataFrame:
    rows = [
        "Ann. Mean (%)",
        "Ann. StdDev (%)",
        "Ann. SemiDev (%)",
        "CVaR 95% (%)",
        "Avg DD (%)",
        "VaR 95% (%)",
        "Sharpe (ann.)",
        "Sortino (ann.)",
        "Tail-Adj Sharpe (CVaR95)",
        "Tail-Adj Sharpe (mVaR95)",
    ]

    out = pd.DataFrame(index=rows)
    for name, r in portfolios.items():
        st = portfolio_stats_paper_style(r, periods_per_year=periods_per_year,
                                         rf_annual=rf_annual, target=target, alpha=alpha)
        out[name] = [st[k] for k in rows]
    return out


In [5]:

# We have daily input--> make it monthly
# NOTE: Industry portfolio is in pct, when we created the data we divided 100!
def compute_monthly_factor_returns_from_daily(daily_factors: pd.DataFrame):
    # as before
    x = daily_factors.copy().dropna(how="all")
    month = x.index.to_period("M")
    monthly = (1.0 + x).groupby(month).prod() - 1.0
    monthly.index = monthly.index.to_timestamp("M")
    return monthly.sort_index()


def gmv_simple(
    daily_ret: pd.DataFrame,
    cols=None,
    ridge: float = 1e-10,
    sum_to_one_constraint: bool = True,  # True => enforce sum(w)=1; False => no equality constraint
    long_only: bool = True,              # True => bounds [0,1]; False => no bounds
    maxiter: int = 10000,
):
    if cols is not None:
        daily_ret = daily_ret[cols]
    # daily input to monthly
    Rm_df = compute_monthly_factor_returns_from_daily(daily_ret)
    cols = list(Rm_df.columns)

    R = Rm_df.to_numpy()
    T, K = R.shape
    print(f"R shape: T (months) = {T:,},  K (assets) = {K:,}")
    # Monthly cov
    Sigma = np.cov(R, rowvar=False) + ridge * np.eye(K)

    def objective(w):
        w = np.asarray(w, float)
        return float(w @ Sigma @ w)  # GMV: minimize variance

    constraints = []
    if sum_to_one_constraint:
        constraints.append({"type": "eq", "fun": lambda w: np.sum(w) - 1.0})

    if long_only:
        bounds = [(0.0, 1)] * K
    else:
        bounds = None  # not bounding is false => no bounds at all

    x0 = np.ones(K) / K  # simple start

    res = minimize(
        objective,
        x0=x0,
        method="SLSQP",
        bounds=bounds,
        constraints=constraints,
        options={"maxiter": maxiter},
    )

    w = np.asarray(res.x, float)
    w = pd.Series(w, index=cols, name="w_gmv")

    # monthly portfolio return series (gross, no cost)
    port_monthly = pd.Series(Rm_df.to_numpy() @ w.to_numpy(), index=Rm_df.index, name="port_gmv")

    return {
        "weights": w,
        "monthly_returns": Rm_df,
        "port_monthly": port_monthly,
        "Sigma": Sigma,
        "opt_result": res,
        "sum_to_one_constraint": sum_to_one_constraint,
        "long_only": long_only,
    }


def mv_simple(
    daily_ret: pd.DataFrame,
    cols=None,
    gamma = 5.0,
    ridge: float = 1e-10,
    sum_to_one_constraint: bool = True,  # True => enforce sum(w)=1; False => no equality constraint
    long_only: bool = True,              # True => bounds [0,inf); False => no bounds
    maxiter: int = 10000,
):
    if cols is not None:
        daily_ret = daily_ret[cols]
    # monthly
    Rm_df = compute_monthly_factor_returns_from_daily(daily_ret)
    cols = list(Rm_df.columns)

    R = Rm_df.to_numpy()
    T, K = R.shape
    print(f"R shape: T (months) = {T:,},  K (assets) = {K:,}")

    Sigma = np.cov(R, rowvar=False) + ridge * np.eye(K)
    Return = np.mean(R, axis=0)

    def objective(w):
        w = np.asarray(w, float)
        return float(0.5 * gamma * (w @ Sigma @ w) - (w @ Return))

    constraints = []
    if sum_to_one_constraint:
        constraints.append({"type": "eq", "fun": lambda w: np.sum(w) - 1.0})

    if long_only:
        bounds = [(0.0, 1)] * K
    else:
        bounds = None  # not bounding is false => no bounds at all

    x0 = np.ones(K) / K  # simple start

    res = minimize(
        objective,
        x0=x0,
        method="SLSQP",
        bounds=bounds,
        constraints=constraints,
        options={"maxiter": maxiter},
    )

    w = np.asarray(res.x, float)
    w = pd.Series(w, index=cols, name="w_mv")

    # monthly portfolio return series (gross, no cost)
    port_monthly = pd.Series(Rm_df.to_numpy() @ w.to_numpy(), index=Rm_df.index, name="port_mv")

    return {
        "weights": w,
        "monthly_returns": Rm_df,
        "port_monthly": port_monthly,
        "Sigma": Sigma,
        "opt_result": res,
        "sum_to_one_constraint": sum_to_one_constraint,
        "long_only": long_only,
    }

# Run GMV and MV simple optimizations
res_mv_no_package = mv_simple(industry, cols=industry.columns, sum_to_one_constraint=True,gamma=5.0, long_only=True)

res_gmv = gmv_simple(industry, cols=industry.columns, sum_to_one_constraint=True, long_only=True)


R shape: T (months) = 105,  K (assets) = 10
R shape: T (months) = 105,  K (assets) = 10


In [6]:
# GLOBAL MINIMUM
W_monthly_res_gmv = res_gmv["weights"]
# add the dates!!! Align with the paper
def daily_portfolio_returns_constant_weights(
    daily_returns: pd.DataFrame,
    weights,
    start: pd.Timestamp = None,
    end: pd.Timestamp = None,
    dropna: str = "any") -> pd.Series:

    R = daily_returns.copy()
    if start is not None or end is not None:
        R = R.loc[slice(start, end)]

    if isinstance(weights, pd.Series):
        w = weights.copy()
    else:
        w = pd.Series(np.asarray(weights, float), index=R.columns)

    # Keep common columns only
    cols = [c for c in R.columns if c in w.index]
    R = R[cols]
    w = w[cols].astype(float)

    # Handle missing values
    R = R.dropna(how=dropna)

    # Portfolio returns
    rp = R.to_numpy() @ w.to_numpy()
    return pd.Series(rp, index=R.index, name="portfolio_return")

# DAILY portfolio returns: to match the tables
START = pd.Timestamp("2016-05-01") # match it
END   = pd.Timestamp("2022-12-31")
asset_cols = [c for c in industry.columns]
port_ret_daily_res_GMV_simple = daily_portfolio_returns_constant_weights(
    industry[asset_cols],
    W_monthly_res_gmv,
    start=START,
    end=END
)
port_ret_daily_res_GMV_simple.name = "GMV_simple"

ports = {"Global Minimum": port_ret_daily_res_GMV_simple}
tbl = make_table_for_portfolios(ports, periods_per_year=252, rf_annual=0.0, target=0.0, alpha=0.95)
print(tbl.round(3))


                          Global Minimum
Ann. Mean (%)                      9.460
Ann. StdDev (%)                   16.228
Ann. SemiDev (%)                  17.564
CVaR 95% (%)                      -2.446
Avg DD (%)                         3.748
VaR 95% (%)                       -1.455
Sharpe (ann.)                      0.583
Sortino (ann.)                     0.539
Tail-Adj Sharpe (CVaR95)           3.867
Tail-Adj Sharpe (mVaR95)           6.895


In [7]:
latex_str = df_to_booktabs_latex(tbl, caption=None, label=None)
with open("portfolio_stats_transposed.tex", "w") as f:
    f.write(latex_str)

print(latex_str)

\begin{tabular}{lr}
\toprule
 & Global Minimum \\
\midrule
Ann. Mean (\%) & 9.46 \\
Ann. StdDev (\%) & 16.23 \\
Ann. SemiDev (\%) & 17.56 \\
CVaR 95\% (\%) & -2.45 \\
Avg DD (\%) & 3.75 \\
VaR 95\% (\%) & -1.46 \\
Sharpe (ann.) & 0.58 \\
Sortino (ann.) & 0.54 \\
Tail-Adj Sharpe (CVaR95) & 3.87 \\
Tail-Adj Sharpe (mVaR95) & 6.89 \\
\bottomrule
\end{tabular}



In [8]:
# with transaction cost logic, we should ignore
def compute_monthly_factor_returns_from_daily(daily_factors: pd.DataFrame):
    # as before
    x = daily_factors.copy().dropna(how="all")
    month = x.index.to_period("M")
    monthly = (1.0 + x).groupby(month).prod() - 1.0
    monthly.index = monthly.index.to_timestamp("M")
    return monthly.sort_index()


def gmv_with_drift_tc_in_objective(
    daily_ret: pd.DataFrame,
    cols=None,
    gamma=5,
    ridge: float = 1e-10,
    use_tc: bool = True,
    c_tc: float = 0.025,
    abs_eps: float = 1e-10,
    long_only: bool = True,
    w_min: float = -0.1,
    w_max: float = 1.5,
    normalize_weights_constraint: bool = True,  # True: normalize AFTER (vol-mgmt logic); False: sum-to-1 constraint
    maxiter: int = 10000):
    
    if cols is not None:
        daily_ret = daily_ret[cols]

    Rm_df = compute_monthly_factor_returns_from_daily(daily_ret)
    cols = list(Rm_df.columns)
    R = Rm_df.to_numpy()
    T, K = R.shape
    print(f"R shape: T (months) = {T:,},  K (assets) = {K:,}")
    Sigma = np.cov(R, rowvar=False) + ridge * np.eye(K)

    def smooth_abs(x):
        return np.sqrt(x * x + abs_eps)

    def normalize(w_raw):
        s = float(np.sum(w_raw))
        if abs(s) < 1e-12:
            return None
        return w_raw / s

    def drift_turnover_series(w_target):
        taus = np.zeros(T, dtype=float)
        if T <= 1:
            return taus
        for t in range(1, T):
            R_prev = R[t - 1]
            g = 1.0 + R_prev
            numer = w_target * g
            denom = float(np.sum(numer))
            w_pre = numer / denom if abs(denom) > 1e-12 else w_target
            taus[t] = 0.5 * float(np.sum(smooth_abs(w_target - w_pre)))
        return taus

    def tc_mean(w_target):
        if (not use_tc) or c_tc <= 0.0:
            return 0.0
        w_target_norm=normalize(w_target) # we always calculate over the normalized weights to be consistant
        taus = drift_turnover_series(w_target_norm)

        return c_tc * float(np.mean(taus[1:])) if T > 1 else 0.0

    def objective(w_raw):
        #  we should add gamma here
        if normalize_weights_constraint:
            w = normalize(np.asarray(w_raw, float))

        else:
            w = np.asarray(w_raw, float)

        risk = float(w @ Sigma @ w)
        return 0.5*gamma*risk + tc_mean(w)

    constraints = []
    if not normalize_weights_constraint:
        constraints.append({"type": "eq", "fun": lambda w: np.sum(w) - 1.0}) # we add up to one

    if long_only:
        bounds = [(0.0, 1)] * K
    else:
        bounds = [(None, None)] * K # we can impose short constraints w_min, w_max

    x0 = np.ones(K) if normalize_weights_constraint else (np.ones(K) / K)

    res = minimize(
        objective,
        x0=x0,
        method="SLSQP",
        bounds=bounds,
        constraints=constraints,
        options={"maxiter": maxiter},
    )

    w_raw = np.asarray(res.x, float)
    w_norm = normalize(w_raw) if normalize_weights_constraint else w_raw # if false, then w_norm = w_raw
    # then we are in the normalize via constraint regime. See constraint
    if w_norm is None:
        w_norm = np.ones(K) / K

    taus = drift_turnover_series(w_norm) if (use_tc and c_tc > 0.0) else np.zeros(T, dtype=float)
    costs = (c_tc * taus) if (use_tc and c_tc > 0.0) else np.zeros(T, dtype=float)
    # monthly cost, but again, we need to translate it to daily...
    gross = R @ w_norm
    net = gross - costs

    return {
        "weights_raw": pd.Series(w_raw, index=cols, name="w_raw"),
        "weights_norm": pd.Series(w_norm, index=cols, name="w_norm"),
        "monthly_returns": Rm_df,
        "turnover": pd.Series(taus, index=Rm_df.index, name="turnover"),
        "costs": pd.Series(costs, index=Rm_df.index, name="costs"),
        "port_gross": pd.Series(gross, index=Rm_df.index, name="gross"),
        "port_net": pd.Series(net, index=Rm_df.index, name="net"),
        "TC_in_objective": float(np.mean(costs[1:])) if T > 1 else 0.0,
        "opt_result": res,
        "use_tc": use_tc,
        "normalize_weights_constraint": normalize_weights_constraint,
    }


res_global_tc = gmv_with_drift_tc_in_objective(
    daily_ret=industry,
    cols=industry.columns,
    use_tc=True,
    gamma=5.0,
    c_tc=0.015,
    long_only=True, # more consistant with RL but not that much with Volatility management cuase there b was unbounded
    normalize_weights_constraint=False
)


R shape: T (months) = 105,  K (assets) = 10


In [9]:
res_global_tc.keys()

dict_keys(['weights_raw', 'weights_norm', 'monthly_returns', 'turnover', 'costs', 'port_gross', 'port_net', 'TC_in_objective', 'opt_result', 'use_tc', 'normalize_weights_constraint'])

In [10]:
res_global_tc['TC_in_objective']

0.00013354010688440622

In [11]:
res_global_tc['costs']

date
2015-12-31    0.000000
2016-01-31    0.000052
2016-02-29    0.000377
2016-03-31    0.000064
2016-04-30    0.000107
                ...   
2024-04-30    0.000144
2024-05-31    0.000216
2024-06-30    0.000190
2024-07-31    0.000187
2024-08-31    0.000136
Freq: ME, Name: costs, Length: 105, dtype: float64

In [12]:
W_monthly_res_global_tc = res_global_tc["weights_norm"]
# add the dates!!! Align with the paper
def daily_portfolio_returns_constant_weights(
    daily_returns: pd.DataFrame,
    weights,
    start: pd.Timestamp = None,
    end: pd.Timestamp = None,
    dropna: str = "any") -> pd.Series:

    R = daily_returns.copy()
    if start is not None or end is not None:
        R = R.loc[slice(start, end)]

    if isinstance(weights, pd.Series):
        w = weights.copy()
    else:
        w = pd.Series(np.asarray(weights, float), index=R.columns)

    # Keep common columns only
    cols = [c for c in R.columns if c in w.index]
    R = R[cols]
    w = w[cols].astype(float)

    # Handle missing values
    R = R.dropna(how=dropna)

    # Portfolio returns
    rp = R.to_numpy() @ w.to_numpy()
    return pd.Series(rp, index=R.index, name="portfolio_return")

# DAILY portfolio returns: to match the tables
START = pd.Timestamp("2016-05-01") # match it
END   = pd.Timestamp("2022-12-31")
asset_cols = [c for c in industry.columns]
port_ret_daily_res_global_tc = daily_portfolio_returns_constant_weights(
    industry[asset_cols],
    W_monthly_res_global_tc,
    start=START,
    end=END
)
port_ret_daily_res_global_tc.name = "GMV_c_015"

ports = {"GMV dynamic cost 0.015": port_ret_daily_res_global_tc}
tbl = make_table_for_portfolios(ports, periods_per_year=252, rf_annual=0.0, target=0.0, alpha=0.95)
print(tbl.round(3))


                          GMV dynamic cost 0.015
Ann. Mean (%)                              9.572
Ann. StdDev (%)                           16.308
Ann. SemiDev (%)                          17.591
CVaR 95% (%)                              -2.441
Avg DD (%)                                 3.774
VaR 95% (%)                               -1.423
Sharpe (ann.)                              0.587
Sortino (ann.)                             0.544
Tail-Adj Sharpe (CVaR95)                   3.922
Tail-Adj Sharpe (mVaR95)                   7.035


### Mean–Variance portfolio with risk aversion ($\gamma=5$) (long-only)

Given expected returns $\mu \in \mathbb{R}^n$ and covariance matrix $\Sigma \in \mathbb{R}^{n\times n}$, the call `max_quadratic_utility(risk_aversion=5)` solves:

$$
\begin{aligned}
\max_{w \in \mathbb{R}^n}\quad 
& \mu^\top w \;-\; \frac{\gamma}{2}\, w^\top \Sigma w \\
\text{s.t.}\quad 
& \mathbf{1}^\top w = 1, \\
& w \ge 0 \qquad \text{(long-only)}.
\end{aligned}
$$



In [14]:
# With Package

industry_data = pd.read_csv(r'c:\Users\95att\Desktop\job\First_paper_QAC\industry_mkt.csv', index_col=0, parse_dates=True) # daily
industry_data = industry_data[[c for c in industry_data.columns if c !='Mkt-RF']] # dont need Mkt-RF

print(industry_data.head())
print("\nColumn names:")
print(industry_data.columns.tolist())

# Use this data for optimization
returns_data = industry_data.copy()
# MONTHLY
returns_monthly = compute_monthly_factor_returns_from_daily(returns_data)

# Calculate expected returns and covariance
# We ll get different results from the without package part cause of the annualization frequency
mu = mean_historical_return(returns_monthly, returns_data=True, frequency=12) # we have monthly data
S = sample_cov(returns_monthly, returns_data=True, frequency=12)

print("\nExpected Annual Returns:")
print(mu.sort_values(ascending=False))
print(returns_monthly.head())

             Durbl   Enrgy   HiTec    Hlth   Manuf   NoDur   Other   Shops  \
date                                                                         
2015-12-14 -0.0028  0.0076  0.0050  0.0040 -0.0016  0.0054 -0.0005  0.0077   
2015-12-15  0.0097  0.0291  0.0041  0.0173  0.0035  0.0089  0.0193 -0.0006   
2015-12-16  0.0266 -0.0070  0.0140  0.0152  0.0141  0.0183  0.0159  0.0153   
2015-12-17 -0.0222 -0.0244 -0.0147 -0.0135 -0.0149 -0.0125 -0.0150 -0.0145   
2015-12-18 -0.0131 -0.0140 -0.0183 -0.0131 -0.0169 -0.0166 -0.0221 -0.0102   

             Telcm   Utils  
date                        
2015-12-14  0.0003  0.0001  
2015-12-15  0.0119  0.0083  
2015-12-16  0.0189  0.0230  
2015-12-17 -0.0137 -0.0040  
2015-12-18 -0.0187 -0.0131  

Column names:
['Durbl', 'Enrgy', 'HiTec', 'Hlth', 'Manuf', 'NoDur', 'Other', 'Shops', 'Telcm', 'Utils']

Expected Annual Returns:
HiTec    0.210100
Durbl    0.151493
Shops    0.140276
Other    0.125415
Manuf    0.115271
Utils    0.105390
Hlth     0.

In [15]:
# No expected target return ! 
gamma = 5
ef_mv = EfficientFrontier(mu, S,weight_bounds=(0.0, 1.0))
ef_mv.add_constraint(lambda w: w >= 0) # make it explicit

weights_mv = ef_mv.max_quadratic_utility(risk_aversion=gamma)
cleaned_weights_mv = ef_mv.clean_weights()

print("Mean-Variance (Max Sharpe) Portfolio Weights:")
print(pd.Series(cleaned_weights_mv).sort_values(ascending=False))

# Get portfolio performance
perf_mv = ef_mv.portfolio_performance(verbose=True, risk_free_rate=0.0)

Mean-Variance (Max Sharpe) Portfolio Weights:
HiTec    0.88453
Utils    0.11547
Durbl    0.00000
Enrgy    0.00000
Hlth     0.00000
Manuf    0.00000
NoDur    0.00000
Other    0.00000
Shops    0.00000
Telcm    0.00000
dtype: float64
Expected annual return: 19.8%
Annual volatility: 18.0%
Sharpe Ratio: 1.10


In [16]:
W_monthly_mv = pd.Series(cleaned_weights_mv)

def daily_portfolio_returns_constant_weights(
    daily_returns: pd.DataFrame,
    weights,
    start: pd.Timestamp = None,
    end: pd.Timestamp = None,
    dropna: str = "any") -> pd.Series:

    R = daily_returns.copy()
    if start is not None or end is not None:
        R = R.loc[slice(start, end)]

    if isinstance(weights, pd.Series):
        w = weights.copy()
    else:
        w = pd.Series(np.asarray(weights, float), index=R.columns)

    # Keep common columns only
    cols = [c for c in R.columns if c in w.index]
    R = R[cols]
    w = w[cols].astype(float)

    R = R.dropna(how=dropna)

    # Portfolio returns
    rp = R.to_numpy() @ w.to_numpy()
    return pd.Series(rp, index=R.index, name="portfolio_return")

# DAILY portfolio returns: to match the tables
START = pd.Timestamp("2016-05-01") # match it
END   = pd.Timestamp("2022-12-31")
asset_cols = [c for c in industry.columns]
port_ret_daily_MV = daily_portfolio_returns_constant_weights(
    industry[asset_cols],
    W_monthly_mv,
    start=START,
    end=END
)
port_ret_daily_MV.name = "MV"

ports = {"MV": port_ret_daily_MV}
tbl = make_table_for_portfolios(ports, periods_per_year=252, rf_annual=0.0, target=0.0, alpha=0.95)
print(tbl.round(3))

                              MV
Ann. Mean (%)             18.264
Ann. StdDev (%)           23.022
Ann. SemiDev (%)          24.943
CVaR 95% (%)              -3.518
Avg DD (%)                 6.683
VaR 95% (%)               -2.278
Sharpe (ann.)              0.793
Sortino (ann.)             0.732
Tail-Adj Sharpe (CVaR95)   5.192
Tail-Adj Sharpe (mVaR95)   8.433


In [17]:
# No package


W_monthly_res_mv_no_package = res_mv_no_package["weights"] # we defined in the beginning with res_gvm
# add the dates!!! Align with the paper
def daily_portfolio_returns_constant_weights(
    daily_returns: pd.DataFrame,
    weights,
    start: pd.Timestamp = None,
    end: pd.Timestamp = None,
    dropna: str = "any") -> pd.Series:

    R = daily_returns.copy()
    if start is not None or end is not None:
        R = R.loc[slice(start, end)]

    if isinstance(weights, pd.Series):
        w = weights.copy()
    else:
        w = pd.Series(np.asarray(weights, float), index=R.columns)

    # Keep common columns only
    cols = [c for c in R.columns if c in w.index]
    R = R[cols]
    w = w[cols].astype(float)

    R = R.dropna(how=dropna)

    # Portfolio returns
    rp = R.to_numpy() @ w.to_numpy()
    return pd.Series(rp, index=R.index, name="portfolio_return")

# DAILY portfolio returns: to match the tables
START = pd.Timestamp("2016-05-01") # match it
END   = pd.Timestamp("2022-12-31")
asset_cols = [c for c in industry.columns]
port_ret_daily_res_MV_simple_no_package = daily_portfolio_returns_constant_weights(
    industry[asset_cols],
    W_monthly_res_mv_no_package,
    start=START,
    end=END
)
port_ret_daily_res_MV_simple_no_package.name = "MV_simple"

ports = {"MV simple": port_ret_daily_res_MV_simple_no_package}
tbl = make_table_for_portfolios(ports, periods_per_year=252, rf_annual=0.0, target=0.0, alpha=0.95)
print(tbl.round(3))


                          MV simple
Ann. Mean (%)                17.982
Ann. StdDev (%)              22.621
Ann. SemiDev (%)             24.426
CVaR 95% (%)                 -3.454
Avg DD (%)                    6.480
VaR 95% (%)                  -2.237
Sharpe (ann.)                 0.795
Sortino (ann.)                0.736
Tail-Adj Sharpe (CVaR95)      5.206
Tail-Adj Sharpe (mVaR95)      8.513


In [18]:
latex_str = df_to_booktabs_latex(tbl, caption=None, label=None)
with open("portfolio_stats_transposed.tex", "w") as f:
    f.write(latex_str)

print(latex_str)

\begin{tabular}{lr}
\toprule
 & MV simple \\
\midrule
Ann. Mean (\%) & 17.98 \\
Ann. StdDev (\%) & 22.62 \\
Ann. SemiDev (\%) & 24.43 \\
CVaR 95\% (\%) & -3.45 \\
Avg DD (\%) & 6.48 \\
VaR 95\% (\%) & -2.24 \\
Sharpe (ann.) & 0.79 \\
Sortino (ann.) & 0.74 \\
Tail-Adj Sharpe (CVaR95) & 5.21 \\
Tail-Adj Sharpe (mVaR95) & 8.51 \\
\bottomrule
\end{tabular}



### Mean–CVaR  portfolio with risk aversion
SGD comment: Use tail sensitive object<br>
Given returns $r_t\in\mathbb{R}^n$ for $t=1,\dots,T$ (rows of $R\in\mathbb{R}^{T\times n}$) and expected returns $\mu\in\mathbb{R}^n$, choose portfolio weights $w\in\mathbb{R}^n$ and auxiliary variables $\alpha\in\mathbb{R}$, $u_t\ge 0$.

Define portfolio loss in scenario $t$ as $\ell_t(w) = -r_t^\top w$.

$$
\begin{aligned}
\min_{w,\alpha,u}\quad
& \gamma\left(\alpha + \frac{1}{(1-\beta)T}\sum_{t=1}^T u_t\right) - \mu^\top w \\
\text{s.t.}\quad
& \mathbf{1}^\top w = 1, \\
& w \ge 0, \\
& u_t \ge 0,\qquad t=1,\dots,T, \\
& u_t \ge -r_t^\top w - \alpha,\qquad t=1,\dots,T .
\end{aligned}
$$

COMMENT: We do 2 exercises, simple CVAR minimization and Markowitz with CVAR (without risk aversion) maximization ( instead of variance)<br>
Here $\beta$ is the CVaR confidence level (e.g. $0.95$) and $\gamma=1$ is the risk-aversion parameter, which is 1 here cause of the cvar .<br>
Source: [PyPortfolioOpt: General Efficient Frontier](https://pyportfolioopt.readthedocs.io/en/stable/GeneralEfficientFrontier.html)



In [20]:

# No package monthly

industry_data = pd.read_csv(r'c:\Users\95att\Desktop\job\First_paper_QAC\industry_mkt.csv', index_col=0, parse_dates=True) # daily
industry_data = industry_data[[c for c in industry_data.columns if c !='Mkt-RF']] # dont need Mkt-RF

print(industry_data.head())
print("\nColumn names:")
print(industry_data.columns.tolist())

# Use this data for optimization
returns_data = industry_data.copy()
# MONTHLY
returns_monthly = compute_monthly_factor_returns_from_daily(returns_data)
R = returns_monthly.values          
mu_vec = np.mean(R, axis=0)
print(f"Monthly returns:{returns_monthly.head()}")

T, n = R.shape
beta = 0.95
gamma = 1.0   

w = cp.Variable(n)
alpha = cp.Variable()     # VaR-like threshold
u = cp.Variable(T)        # tail slack variables: see documentation

losses = -(R @ w)         # loss = -portfolio return
cvar = alpha + (1 / ((1 - beta) * T)) * cp.sum(u)

constraints = [
    cp.sum(w) == 1,
    w >= 0,               # long-only 
    u >= 0,
    u >= losses - alpha
]

objective = cp.Minimize(gamma * cvar - mu_vec @ w)

prob = cp.Problem(objective, constraints)
prob.solve(solver="ECOS")

MV_Cvar_weights = pd.Series(w.value, index=industry_data.columns).sort_values(ascending=False)
print(MV_Cvar_weights)
print("Expected return:", float(mu_vec @ w.value))
print("CVaR(loss):", float(cvar.value))


             Durbl   Enrgy   HiTec    Hlth   Manuf   NoDur   Other   Shops  \
date                                                                         
2015-12-14 -0.0028  0.0076  0.0050  0.0040 -0.0016  0.0054 -0.0005  0.0077   
2015-12-15  0.0097  0.0291  0.0041  0.0173  0.0035  0.0089  0.0193 -0.0006   
2015-12-16  0.0266 -0.0070  0.0140  0.0152  0.0141  0.0183  0.0159  0.0153   
2015-12-17 -0.0222 -0.0244 -0.0147 -0.0135 -0.0149 -0.0125 -0.0150 -0.0145   
2015-12-18 -0.0131 -0.0140 -0.0183 -0.0131 -0.0169 -0.0166 -0.0221 -0.0102   

             Telcm   Utils  
date                        
2015-12-14  0.0003  0.0001  
2015-12-15  0.0119  0.0083  
2015-12-16  0.0189  0.0230  
2015-12-17 -0.0137 -0.0040  
2015-12-18 -0.0187 -0.0131  

Column names:
['Durbl', 'Enrgy', 'HiTec', 'Hlth', 'Manuf', 'NoDur', 'Other', 'Shops', 'Telcm', 'Utils']
Monthly returns:               Durbl     Enrgy     HiTec      Hlth     Manuf     NoDur  \
date                                                   

In [21]:
W_monthly_MV_Cvar_weights = pd.Series(MV_Cvar_weights)

def daily_portfolio_returns_constant_weights(
    daily_returns: pd.DataFrame,
    weights,
    start: pd.Timestamp = None,
    end: pd.Timestamp = None,
    dropna: str = "any") -> pd.Series:

    R = daily_returns.copy()
    if start is not None or end is not None:
        R = R.loc[slice(start, end)]

    if isinstance(weights, pd.Series):
        w = weights.copy()
    else:
        w = pd.Series(np.asarray(weights, float), index=R.columns)

    # Keep common columns only
    cols = [c for c in R.columns if c in w.index]
    R = R[cols]
    w = w[cols].astype(float)

    # Handle missing values
    R = R.dropna(how=dropna)

    # Portfolio returns
    rp = R.to_numpy() @ w.to_numpy()
    return pd.Series(rp, index=R.index, name="portfolio_return")

# DAILY portfolio returns: to match the tables
START = pd.Timestamp("2016-05-01") # match it
END   = pd.Timestamp("2022-12-31")
asset_cols = [c for c in industry.columns]
port_ret_daily_cvar = daily_portfolio_returns_constant_weights(
    industry[asset_cols],
    W_monthly_MV_Cvar_weights,
    start=START,
    end=END
)
port_ret_daily_cvar.name = "MV_Cvar_weights"

ports = {"MV_Cvar_weights": port_ret_daily_cvar}
tbl = make_table_for_portfolios(ports, periods_per_year=252, rf_annual=0.0, target=0.0, alpha=0.95)
print(tbl.round(3))

                          MV_Cvar_weights
Ann. Mean (%)                      11.301
Ann. StdDev (%)                    16.994
Ann. SemiDev (%)                   17.787
CVaR 95% (%)                       -2.491
Avg DD (%)                          3.865
VaR 95% (%)                        -1.509
Sharpe (ann.)                       0.665
Sortino (ann.)                      0.635
Tail-Adj Sharpe (CVaR95)            4.537
Tail-Adj Sharpe (mVaR95)            7.767


In [22]:
latex_str = df_to_booktabs_latex(tbl, caption=None, label=None)
with open("portfolio_stats_transposed.tex", "w") as f:
    f.write(latex_str)

print(latex_str)

\begin{tabular}{lr}
\toprule
 & MV\_Cvar\_weights \\
\midrule
Ann. Mean (\%) & 11.30 \\
Ann. StdDev (\%) & 16.99 \\
Ann. SemiDev (\%) & 17.79 \\
CVaR 95\% (\%) & -2.49 \\
Avg DD (\%) & 3.87 \\
VaR 95\% (\%) & -1.51 \\
Sharpe (ann.) & 0.66 \\
Sortino (ann.) & 0.64 \\
Tail-Adj Sharpe (CVaR95) & 4.54 \\
Tail-Adj Sharpe (mVaR95) & 7.77 \\
\bottomrule
\end{tabular}



In [23]:
# With Package, only CVAR
returns_data = industry_data.copy()
returns_monthly = compute_monthly_factor_returns_from_daily(returns_data)
print(f"Monthly returns:{returns_monthly.head()}")

# Calculate expected returns and covariance
mu = mean_historical_return(returns_monthly, returns_data=True, frequency=12) # we have daily data
S = sample_cov(returns_monthly, returns_data=True, frequency=12)

ef_cvar = EfficientCVaR(mu, returns_monthly, beta=0.95,solver='ECOS')
weights_cvar = ef_cvar.min_cvar()
cleaned_weights_cvar = ef_cvar.clean_weights()

print("CVaR (95% confidence) Portfolio Weights:")
print(pd.Series(cleaned_weights_cvar).sort_values(ascending=False))

# Get portfolio performance
perf_cvar = ef_cvar.portfolio_performance(verbose=True)

Monthly returns:               Durbl     Enrgy     HiTec      Hlth     Manuf     NoDur  \
date                                                                     
2015-12-31  0.022053  0.008738  0.004718  0.030117  0.006933  0.022754   
2016-01-31 -0.123572 -0.043696 -0.059695 -0.101357 -0.057026  0.001530   
2016-02-29  0.040787 -0.029230 -0.008662 -0.009855  0.033711  0.005770   
2016-03-31  0.106740  0.109014  0.084943  0.039770  0.069228  0.049924   
2016-04-30  0.015581  0.091661 -0.042075  0.049705  0.023032  0.002650   

               Other     Shops     Telcm     Utils  
date                                                
2015-12-31  0.016382  0.017000  0.008563  0.039333  
2016-01-31 -0.087496 -0.045149  0.003535  0.046675  
2016-02-29 -0.004241  0.001539  0.009236  0.016958  
2016-03-31  0.071445  0.055490  0.061620  0.078541  
2016-04-30  0.024171 -0.005340  0.007275 -0.007957  
CVaR (95% confidence) Portfolio Weights:
Hlth     0.63544
Utils    0.33969
NoDur    0.02487
Du

In [24]:
W_monthly_cleaned_weights_cvar = pd.Series(cleaned_weights_cvar)

def daily_portfolio_returns_constant_weights(
    daily_returns: pd.DataFrame,
    weights,
    start: pd.Timestamp = None,
    end: pd.Timestamp = None,
    dropna: str = "any") -> pd.Series:

    R = daily_returns.copy()
    if start is not None or end is not None:
        R = R.loc[slice(start, end)]

    if isinstance(weights, pd.Series):
        w = weights.copy()
    else:
        w = pd.Series(np.asarray(weights, float), index=R.columns)

    # Keep common columns only
    cols = [c for c in R.columns if c in w.index]
    R = R[cols]
    w = w[cols].astype(float)

    # Handle missing values
    R = R.dropna(how=dropna)

    # Portfolio returns
    rp = R.to_numpy() @ w.to_numpy()
    return pd.Series(rp, index=R.index, name="portfolio_return")

# DAILY portfolio returns: to match the tables
START = pd.Timestamp("2016-05-01") # match it
END   = pd.Timestamp("2022-12-31")
asset_cols = [c for c in industry.columns]
port_ret_daily_cvar = daily_portfolio_returns_constant_weights(
    industry[asset_cols],
    W_monthly_cleaned_weights_cvar,
    start=START,
    end=END
)
port_ret_daily_cvar.name = "CVaR_min"

ports = {"CVaR Minimization": port_ret_daily_cvar}
tbl = make_table_for_portfolios(ports, periods_per_year=252, rf_annual=0.0, target=0.0, alpha=0.95)
print(tbl.round(3))

                          CVaR Minimization
Ann. Mean (%)                        11.307
Ann. StdDev (%)                      16.985
Ann. SemiDev (%)                     17.814
CVaR 95% (%)                         -2.490
Avg DD (%)                            3.889
VaR 95% (%)                          -1.509
Sharpe (ann.)                         0.666
Sortino (ann.)                        0.635
Tail-Adj Sharpe (CVaR95)              4.541
Tail-Adj Sharpe (mVaR95)              7.715


In [25]:
# without package only cvar
print(f"Monthly returns:{returns_monthly.head()}")

R = returns_monthly.values          
mu_vec = np.mean(R, axis=0)

T, n = R.shape
beta = 0.95
gamma = 1.0 

w = cp.Variable(n)
alpha = cp.Variable()     # VaR-like threshold
u = cp.Variable(T)        # tail slack variables: see documentation

losses = -(R @ w)         # loss = -portfolio return
cvar = alpha + (1 / ((1 - beta) * T)) * cp.sum(u)

constraints = [
    cp.sum(w) == 1,
    w >= 0,               # long-only 
    u >= 0,
    u >= losses - alpha
]

objective = cp.Minimize(gamma * cvar)

prob = cp.Problem(objective, constraints)
prob.solve(solver="ECOS")

Cvar_weights_no_package = pd.Series(w.value, index=industry_data.columns).sort_values(ascending=False)
print(Cvar_weights_no_package)
print("Expected return:", float(mu_vec @ w.value))
print("CVaR(loss):", float(cvar.value))


Monthly returns:               Durbl     Enrgy     HiTec      Hlth     Manuf     NoDur  \
date                                                                     
2015-12-31  0.022053  0.008738  0.004718  0.030117  0.006933  0.022754   
2016-01-31 -0.123572 -0.043696 -0.059695 -0.101357 -0.057026  0.001530   
2016-02-29  0.040787 -0.029230 -0.008662 -0.009855  0.033711  0.005770   
2016-03-31  0.106740  0.109014  0.084943  0.039770  0.069228  0.049924   
2016-04-30  0.015581  0.091661 -0.042075  0.049705  0.023032  0.002650   

               Other     Shops     Telcm     Utils  
date                                                
2015-12-31  0.016382  0.017000  0.008563  0.039333  
2016-01-31 -0.087496 -0.045149  0.003535  0.046675  
2016-02-29 -0.004241  0.001539  0.009236  0.016958  
2016-03-31  0.071445  0.055490  0.061620  0.078541  
2016-04-30  0.024171 -0.005340  0.007275 -0.007957  
Hlth     6.354419e-01
Utils    3.396851e-01
NoDur    2.487303e-02
Enrgy    1.501868e-11
Telcm 

In [26]:
W_monthly_Cvar_weights_no_package = pd.Series(Cvar_weights_no_package)

def daily_portfolio_returns_constant_weights(
    daily_returns: pd.DataFrame,
    weights,
    start: pd.Timestamp = None,
    end: pd.Timestamp = None,
    dropna: str = "any") -> pd.Series:

    R = daily_returns.copy()
    if start is not None or end is not None:
        R = R.loc[slice(start, end)]

    if isinstance(weights, pd.Series):
        w = weights.copy()
    else:
        w = pd.Series(np.asarray(weights, float), index=R.columns)

    # Keep common columns only
    cols = [c for c in R.columns if c in w.index]
    R = R[cols]
    w = w[cols].astype(float)

    # Handle missing values
    R = R.dropna(how=dropna)

    # Portfolio returns
    rp = R.to_numpy() @ w.to_numpy()
    return pd.Series(rp, index=R.index, name="portfolio_return")

# DAILY portfolio returns: to match the tables
START = pd.Timestamp("2016-05-01") # match it
END   = pd.Timestamp("2022-12-31")
asset_cols = [c for c in industry.columns]
port_ret_daily_cvar = daily_portfolio_returns_constant_weights(
    industry[asset_cols],
    W_monthly_Cvar_weights_no_package,
    start=START,
    end=END
)
port_ret_daily_cvar.name = "CVaR_min"

ports = {"CVaR No Package": port_ret_daily_cvar}
tbl = make_table_for_portfolios(ports, periods_per_year=252, rf_annual=0.0, target=0.0, alpha=0.95)
print(tbl.round(3))

                          CVaR No Package
Ann. Mean (%)                      11.307
Ann. StdDev (%)                    16.985
Ann. SemiDev (%)                   17.814
CVaR 95% (%)                       -2.490
Avg DD (%)                          3.889
VaR 95% (%)                        -1.509
Sharpe (ann.)                       0.666
Sortino (ann.)                      0.635
Tail-Adj Sharpe (CVaR95)            4.541
Tail-Adj Sharpe (mVaR95)            7.715


In [27]:
latex_str = df_to_booktabs_latex(tbl, caption=None, label=None)
with open("portfolio_stats_transposed.tex", "w") as f:
    f.write(latex_str)

print(latex_str)

\begin{tabular}{lr}
\toprule
 & CVaR No Package \\
\midrule
Ann. Mean (\%) & 11.31 \\
Ann. StdDev (\%) & 16.99 \\
Ann. SemiDev (\%) & 17.81 \\
CVaR 95\% (\%) & -2.49 \\
Avg DD (\%) & 3.89 \\
VaR 95\% (\%) & -1.51 \\
Sharpe (ann.) & 0.67 \\
Sortino (ann.) & 0.63 \\
Tail-Adj Sharpe (CVaR95) & 4.54 \\
Tail-Adj Sharpe (mVaR95) & 7.71 \\
\bottomrule
\end{tabular}

